In [6]:
import pandas as pd
import os
import glob
import numpy as np
import pandas_ta as ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from yahoo_finance import Share

In [7]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "input_ada/*.csv"))

In [8]:
df_all = pd.DataFrame()
for f in csv_files:
    df = pd.read_csv(f, names=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
    df_all = pd.concat([df,df_all])

In [10]:
df_all

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,1651363200000,0.7555,0.7598,0.7542,0.7597,1288872.2,1651363499999,9.753360e+05,1816,593963.6,449613.96411,0
1,1651363500000,0.7597,0.7613,0.7577,0.7612,1083449.6,1651363799999,8.232017e+05,1521,505000.2,383726.35391,0
2,1651363800000,0.7612,0.7650,0.7582,0.7617,1845440.3,1651364099999,1.406306e+06,2824,1050025.7,799924.41876,0
3,1651364100000,0.7617,0.7626,0.7603,0.7609,989416.0,1651364399999,7.535015e+05,1324,520225.0,396157.67509,0
4,1651364400000,0.7608,0.7615,0.7587,0.7597,955595.0,1651364699999,7.258976e+05,1474,512468.0,389239.04238,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8923,1643672100000,1.0520,1.0550,1.0520,1.0520,242239.9,1643672399999,2.551595e+05,490,117976.9,124277.05080,0
8924,1643672400000,1.0530,1.0530,1.0490,1.0490,369557.4,1643672699999,3.884496e+05,523,146845.4,154397.87860,0
8925,1643672700000,1.0500,1.0510,1.0490,1.0500,174839.3,1643672999999,1.835454e+05,344,99919.3,104916.77400,0
8926,1643673000000,1.0500,1.0520,1.0490,1.0520,345385.7,1643673299999,3.629725e+05,373,239851.3,252097.90170,0


In [11]:
df_all = df_all.sort_values('Open time', ascending=True)

In [12]:
df_all['date'] = pd.to_datetime(df_all['Open time'], unit='ms', origin='unix')

In [13]:
df_all = df_all.set_index('date')

In [14]:
df_all = df_all[df_all.index > '2022-01-01']

In [15]:
df_all['RSI_14'] = df_all.ta.rsi(length=14)

In [16]:
df_all['Signal_long'] = np.where((df_all['RSI_14'] > 50) & (df_all.shift(1)['RSI_14'] < 50), 1 ,0)

In [17]:
df_all['Signal_short'] = np.where((df_all['RSI_14'] < 50) & (df_all.shift(1)['RSI_14'] > 50), 2 ,0)

In [18]:
df_all['Signal'] = df_all[['Signal_long', 'Signal_short']].max(axis=1)

In [19]:
df_all['Signal_long_close'] = np.where((df_all['Signal_long'] > 0).shift(11).fillna(False), 3, 0)

In [20]:
df_all['Signal_short_close'] = np.where((df_all['Signal_short'] > 0).shift(11).fillna(False), 4, 0)

In [21]:
def get_signal():
    return df_all[14:].Signal

In [22]:
def get_signal_longc():
    return df_all[14:].Signal_long_close

In [23]:
def get_signal_shortc():
    return df_all[14:].Signal_short_close

In [24]:
from backtesting import Strategy
class strat3(Strategy):
    def init(self):
        #super().init()
        self.signal1 = self.I(get_signal)
        self.signal2 = self.I(get_signal_longc)
        self.signal3 = self.I(get_signal_shortc)
        self.counter = 0
        
    def next(self):
        #super().next()
        self.counter = self.counter + 1
        #print(self.counter, self.trades)
        if self.signal1==1:
            #print(self.counter, self.trades)
            #sl1 = self.ha_min[-1]
            #tp1 = self.data.ha_close[-1] + 600e-4
            self.buy(size=.10) #sl=sl1) #, tp=tp1)
        if self.signal1==2:
            #print(self.counter, self.trades)
            #sl1 = self.ha_high[-1]
            #tp1 = self.data.Close[-1] - 600e-4
            self.sell(size=.10) #sl=sl1) #, tp=tp1)
        if self.signal2 == 3:
            for i in self.trades[:]:
                if i.is_long:
                    i.close()
                    break
        if self.signal3 == 4:
            for i in self.trades[:]:
                if i.is_short:
                    i.close()
                    break

In [25]:
bt_l = Backtest(df_all[14:], strat3, cash=100_000, commission=.002, hedging=True)  # ,exclusive_orders=True)
stat_l = bt_l.run()
stat_l

Start                     2022-01-01 01:15:00
End                       2022-05-31 23:55:00
Duration                    150 days 22:40:00
Exposure Time [%]                     64.5688
Equity Final [$]                      36107.1
Equity Peak [$]                        100000
Return [%]                           -63.8929
Buy & Hold Return [%]                -53.0838
Return (Ann.) [%]                    -91.3439
Volatility (Ann.) [%]                 0.65019
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                    -63.9133
Avg. Drawdown [%]                    -63.9133
Max. Drawdown Duration      150 days 21:20:00
Avg. Drawdown Duration      150 days 21:20:00
# Trades                                 5564
Win Rate [%]                          39.2164
Best Trade [%]                        7.65088
Worst Trade [%]                      -9.59346
Avg. Trade [%]                    

In [26]:
stat_l._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-7556,17,28,1.323348,1.3270,-27.594512,-0.002760,2022-01-01 02:40:00,2022-01-01 03:35:00,0 days 00:55:00
1,6753,18,29,1.331658,1.3270,-31.455474,-0.003498,2022-01-01 02:45:00,2022-01-01 03:40:00,0 days 00:55:00
2,-6108,19,30,1.325344,1.3280,-16.222848,-0.002004,2022-01-01 02:50:00,2022-01-01 03:45:00,0 days 00:55:00
3,5469,25,36,1.331658,1.3160,-85.633602,-0.011758,2022-01-01 03:20:00,2022-01-01 04:15:00,0 days 00:55:00
4,-4949,26,37,1.324346,1.3160,41.304354,0.006302,2022-01-01 03:25:00,2022-01-01 04:20:00,0 days 00:55:00
...,...,...,...,...,...,...,...,...,...,...
5559,4844,43440,43451,0.604707,0.6115,32.905292,0.011234,2022-05-31 21:15:00,2022-05-31 22:10:00,0 days 00:55:00
5560,-4385,43441,43452,0.600397,0.6087,-36.409532,-0.013830,2022-05-31 21:20:00,2022-05-31 22:15:00,0 days 00:55:00
5561,3896,43442,43453,0.606010,0.6102,16.325798,0.006915,2022-05-31 21:25:00,2022-05-31 22:20:00,0 days 00:55:00
5562,-3534,43445,43456,0.602093,0.6130,-38.543924,-0.018114,2022-05-31 21:40:00,2022-05-31 22:35:00,0 days 00:55:00


In [27]:
stat_l._trades.query('Size > 0').PnL.sum()

-29493.861124400028

In [28]:
stat_l._trades.query('Size < 0').PnL.sum()

-34399.008127200075

In [29]:
stat_l._trades.to_csv('strat3test.csv')

bt_l.run()
bt_l.plot()

# Correlation check about FED QE, Stock price and Inflation

In [30]:
from pandas_datareader.data import DataReader
from datetime import date
series_code = 'FEDFUNDS'
data_source = 'fred'
start = date(2003, 1, 1)
data = DataReader(series_code, data_source, start)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 235 entries, 2003-01-01 to 2022-07-01
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FEDFUNDS  235 non-null    float64
dtypes: float64(1)
memory usage: 3.7 KB


In [31]:
data = data.resample('D').agg('last').fillna(method='ffill')

In [32]:
series_code = 'CORESTICKM159SFRBATL'
data_source = 'fred'
start = date(2003, 1, 1)
data2 = DataReader(series_code, data_source, start)
data2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 235 entries, 2003-01-01 to 2022-07-01
Data columns (total 1 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CORESTICKM159SFRBATL  235 non-null    float64
dtypes: float64(1)
memory usage: 3.7 KB


In [33]:
data2 = data2.resample('D').agg('last').fillna(method='ffill')

In [34]:
import yahoo_fin.stock_info as si
import pandas as pd
import numpy as np
import yfinance as yf

In [35]:
df_og = yf.download('QQQ','2003-01-01', progress=False)

In [36]:
df_og.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2002-12-31,24.540001,24.709999,24.280001,24.370001,21.042534,46015400
2003-01-02,24.719999,25.549999,24.450001,25.400000,21.931908,65903200
2003-01-03,25.459999,25.709999,25.280001,25.680000,22.173668,39059500
2003-01-06,25.709999,26.590000,25.200001,26.320000,22.726288,67223400
2003-01-07,26.410000,26.930000,26.250000,26.650000,23.011230,88726100


In [37]:
df = df_og

In [38]:
df = pd.concat([data,df,data2], axis=1)

In [39]:
df = df.rename(columns={'FEDFUNDS':'Fed_rate', 'CORESTICKM159SFRBATL':'Inflation_rate'})

In [40]:
from pandas.tseries.frequencies import to_offset

logic = {'Open'  : 'first',
         'High'  : 'max',
         'Low'   : 'min',
         'Close' : 'last',
         'Volume': 'sum',
         'Fed_rate' : 'mean',
         'Inflation_rate' : 'mean'}
#df['<DTYYYYMMDD>'] = pd.to_datetime(df['<DTYYYYMMDD>'])
#df = df.set_index('<DTYYYYMMDD>')
df = df.sort_index()
df = df.resample('W').apply(logic)
df.index = df.index - pd.tseries.frequencies.to_offset("6D")


In [41]:
df.corr()

,Open,High,Low,Close,Volume,Fed_rate,Inflation_rate
Open,1.000000,0.999673,0.999567,0.999150,-0.392636,-0.273967,0.330011
High,0.999673,1.000000,0.999433,0.999569,-0.389216,-0.274179,0.332629
Low,0.999567,0.999433,1.000000,0.999549,-0.401644,-0.274170,0.325506
Close,0.999150,0.999569,0.999549,1.000000,-0.397230,-0.274647,0.326253
Volume,-0.392636,-0.389216,-0.401644,-0.397230,1.000000,0.401734,0.170600
Fed_rate,-0.273967,-0.274179,-0.274170,-0.274647,0.401734,1.000000,0.388304
Inflation_rate,0.330011,0.332629,0.325506,0.326253,0.170600,0.388304,1.000000


In [42]:
test = df[['Close', 'Inflation_rate']]

In [43]:
test = test.dropna()

In [44]:
test

,Close,Inflation_rate
2002-12-30,25.680000,2.787553
2003-01-06,27.100000,2.787553
2003-01-13,25.309999,2.787553
2003-01-20,24.820000,2.787553
2003-01-27,24.440001,2.763869
...,...,...
2022-05-30,306.200012,5.297281
2022-06-06,288.839996,5.425963
2022-06-13,274.690002,5.425963
2022-06-20,294.609985,5.425963


In [45]:
from sklearn.model_selection import train_test_split

In [46]:
X_train, X_test, y_train, y_test = train_test_split(test[['Inflation_rate']], test[['Close']], test_size=.2)

In [47]:
from sklearn.linear_model import LinearRegression

In [48]:
# Create Regression Model
model = LinearRegression()
# Train the model
model.fit(X_train, y_train)
# Use model to make predictions
y_pred = model.predict(X_test)

In [49]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# Printout relevant metrics
print("Model Coefficients:", model.coef_)
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("Coefficient of Determination:", r2_score(y_test, y_pred))

Model Coefficients: [[44.20075091]]
Mean Absolute Error: 65.27085880117998
Coefficient of Determination: 0.11594475426264494


In [50]:
X_test

,Inflation_rate
2016-03-21,2.531878
2016-02-08,2.544209
2017-09-18,2.106697
2012-05-21,2.189684
2006-01-16,2.451019
...,...
2019-07-01,2.467932
2015-02-23,1.984445
2012-05-28,2.211568
2006-05-22,2.783959


In [51]:
# Snippet from https://kernc.github.io/backtesting.py/
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.data.Close, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.data.Close):
            self.sell()


bt = Backtest(df, SmaCross,
              cash=100000, commission=.002,
              exclusive_orders=True, trade_on_close=True)

output = bt.run()
bt.plot()

Row(id='1392', ...)

In [52]:
trades = bt.run()

In [53]:
trades

Start                     2002-12-30 00:00:00
End                       2022-08-15 00:00:00
Duration                   7168 days 00:00:00
Exposure Time [%]                     93.8537
Equity Final [$]                      94771.6
Equity Peak [$]                        101944
Return [%]                           -5.22844
Buy & Hold Return [%]                 1193.57
Return (Ann.) [%]                    -1.31158
Volatility (Ann.) [%]                 39.8275
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                    -59.6787
Avg. Drawdown [%]                    -59.6787
Max. Drawdown Duration     6727 days 00:00:00
Avg. Drawdown Duration     6727 days 00:00:00
# Trades                                  104
Win Rate [%]                          29.8077
Best Trade [%]                        38.5613
Worst Trade [%]                      -10.0312
Avg. Trade [%]                    

In [54]:
trades._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2821,62,65,35.438978,37.099998,-4685.737841,-0.046870,2004-03-08,2004-03-29,21 days
1,2563,65,67,37.174198,36.090000,-2778.800290,-0.029165,2004-03-29,2004-04-12,14 days
2,-2569,67,68,36.017820,37.209999,-3062.707677,-0.033100,2004-04-12,2004-04-19,7 days
3,2399,68,69,37.284419,34.770000,-6032.090281,-0.067439,2004-04-19,2004-04-26,7 days
4,-2404,69,73,34.700460,36.549999,-4446.291228,-0.053300,2004-04-26,2004-05-24,28 days
...,...,...,...,...,...,...,...,...,...,...
99,248,902,930,215.720573,269.380005,13307.539039,0.248745,2020-04-13,2020-10-26,196 days
100,-248,930,931,268.841245,294.609985,-6390.647639,-0.095851,2020-10-26,2020-11-02,7 days
101,204,931,992,295.199205,379.859985,17270.799126,0.286792,2020-11-02,2022-01-03,427 days
102,-204,992,1021,379.100265,315.459991,12982.615881,0.167872,2022-01-03,2022-07-25,203 days
